#Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.
https://www.google.com/drive/

https://colab.research.google.com/

##(i) Here we import all necessary libraries
Please put all your files at a desired location. For example, under "/content/drive/MyDrive/Colab Notbooks/USC_BA_estimator/" folder and load our packages. 



In [1]:
from google.colab import drive

drive.mount("/content/drive")
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Path to our Jupyter notebook.
# sys.path.append(
#     os.path.join("/content/drive/MyDrive/test_brain_age/function", "function")
# )
sys.path.append(
    os.path.join("/content/drive/MyDrive/Colab Notebooks/test_brain_age", "function")
)
import nibabel                      as nib
import nibabel.freesurfer.mghformat as mgh
from scipy.ndimage import zoom
from glob import glob
import time
import datetime
import pandas as pd
from preprocess import processmgz
import savepath_updated
import repadding
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
import modelcnn
import sklearn
import saliencymaps
import multiprocessing as mp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##(ii) Main function.


In [2]:
def mainfunction(dir_test_brain_age, path_to_model_weights, path_to_brain_mri):
    # we supposed the brain.mgz files are saved under this format: "/path_to_brain_mri/subject_id/brain.mgz"
    path_of_mgz = os.listdir(path_to_brain_mri)
    brains_tmp = []
    subj_id = []
    for mgz_id in path_of_mgz:
      if os.path.isdir(path_to_brain_mri+"/"+mgz_id):
        subj_id.append(mgz_id)
        mgh_file = mgh.load(path_to_brain_mri+"/"+mgz_id+"/brain.mgz")
        brains_tmp.append(zoom(mgh_file.get_data(), (0.5, 0.5, 0.5)))
    
    brains, coordinates = processmgz(brains_tmp)
    time_split = time.ctime().split(" ")
    (
        brain_save_path,
        coordinates_save_path,
        coordinates_save_path_csv,
    ) = savepath_updated.filename_brainnpy(dir_test_brain_age, time_split)

    # save the cropped brains
    np.save(brain_save_path, brains)

    # save information for repadding.
    np.save(coordinates_save_path, coordinates)
    df_coordinates = pd.DataFrame(coordinates)
    df_coordinates.columns = ["x", "y", "z"]
    df_coordinates["subj_id"] = subj_id
    df_coordinates.to_csv(coordinates_save_path_csv)

    # Build model. 
    model = modelcnn.get_model(width=82, height=86, depth=100)

    # Load best weights.
    model.load_weights(path_to_model_weights)

    # make brain age predictions
    predictions = model.predict(brains) + 22

    # save the prediction results. Of note, the input subject ids should be saved as csv.
    
    BA_save_path_npy, BA_save_path_csv = savepath_updated.filename_pred(dir_test_brain_age, time_split)
    df_BA = pd.DataFrame(predictions)
    df_BA.columns = ["BA"]
    df_BA["subj_id"] = subj_id
    df_BA.to_csv(BA_save_path_csv)

    # Generate saliency mapes
    saliency_map_list = saliencymaps.smap(model, brains)

    # Repadding the saliency maps
    coordinates = np.load(coordinates_save_path)
    repadded_saliency_maps = repadding.repadding(saliency_map_list, coordinates)

    # Save the saliency maps
    smap_path = savepath_updated.filename_smap(dir_test_brain_age, time_split)
    np.save(smap_path, repadded_saliency_maps)
    return df_BA, repadded_saliency_maps, brains


##(iii) Execute main function. 
"dir_test_brain_age" is the path to the folder of all the functions. \\
"path_to_model_weights" is the path to the h5 file. \\
"path_to_brain_mri" is the path to all the brain.mgz files. Here, we supposed the brain.mgz files are saved under this format: "/path_to_brain_mri/subject_id/brain.mgz". \\

The predicted BA will be saved as csv files under "dir_test_brain_age". \\
The csv file is saved in this format "BA-year-month-day-hour:min:sec.csv" \\

The processed saliency maps will be saved under "dir_test_brain_age."\\
The saliency maps will be saved in this format "Saliency_maps-year-month-day-hour:min:sec.csv" \\




In [4]:
# path to the folder
dir_test_brain_age = "/content/drive/MyDrive/Colab Notebooks/test_brain_age"

# path to the h5 file
path_to_model_weights = os.path.join(
    dir_test_brain_age, "function/model/saved-model-317.h5"
)

# path to the brain.mgz files

path_to_brain_mri = os.path.join(
    dir_test_brain_age, "brain_mri"
)

# the mainfunction will automatically save and output the predicted BAs and saliency maps.
predicted_BA, repadded_saliency_maps, brains = mainfunction(
    dir_test_brain_age, path_to_model_weights, path_to_brain_mri
)


<ipython-input-2-6808ab4c2fe1>:10: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  brains_tmp.append(zoom(mgh_file.get_data(), (0.5, 0.5, 0.5)))


1/1 [==============================] - 2s 2s/step


In [7]:
brains.shape, predicted_BA

((2, 82, 86, 100, 1),
           BA     subj_id
 0  83.862534  153_S_4172
 1  72.929840  141_S_0853)